<a href="https://colab.research.google.com/github/JeongCheolHee/colab/blob/main/%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#검증 테스트
#테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법 
#훈련세트 중 20~30%를 떼어 내어 검증 세트로 만듬

In [ ]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [ ]:
#검증 세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target , test_size = 0.2 , random_state = 42)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
#sub_input, sub_target, val_input, val_target을 사용해 모델 제작 및 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state =42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [ ]:
#검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정함-> 교차 검증을 이용
#교차검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있음
#k-겹 교차 검징 : 훈련 세트를 k 부분으로 나눠서 교차 검증을 수행하는 것
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores,'\n')
#교차 검증의 최종 점수는 test_score키에 담긴 5개의 점수를 평균하여 얻을 수 있음(주의 : 이름은 test_score지만 검증 폴드의 점수임)
import numpy as np
print(np.mean(scores['test_score']))

{'fit_time': array([0.01014328, 0.00733471, 0.00612998, 0.00611877, 0.00589657]), 'score_time': array([0.00085354, 0.00061369, 0.00047135, 0.00055122, 0.00053596]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])} 

0.855300214703487


In [ ]:
#주의 : cross_validate 는 train_test_split과 달리 데이터를 섞지 않기 때문에 따로 섞어줘야함
#사이키럿의 분할기는 교차 검증에서 KFold분할기를 사용하기 위해 기본적으로 StratifiedKFold를 사용함
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']), '\n')
#만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하기 위해서는 다음과 같은 코드 작성
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.855300214703487 

0.8574181117533719


In [21]:
#하이퍼파라미터 튜닝
#하이퍼파라미터 : 사용자 지정 파라미터
#튜닝 과정 : 라이브러리가 제공하는 기본값을 그대로 사용해 훈련, 그 다음 검증 세트 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔 가며 훈련
#여러개의 매개변수들이 동시에 만족하는 최적값을 찾아야 하기 때문에 복잡함 -> 그리드 서치 사용
#GridSearchCV클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
#GridSearchCV의 cv 매개변수 기본값은 5 따라서 min_impur~~~값마다 교차검증을 수행하므로 총 25번
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [25]:
#교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야하지만, 그리드 서치는 훈련이 끝나면
#전체 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련함
dt = gs.best_estimator_
print(dt.score(train_input, train_target),'\n')
print(gs.best_params_) # 최적의 하이퍼파라미터
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117 

{'min_impurity_decrease': 0.0001}
{'min_impurity_decrease': 0.0001}


In [26]:
#그리드 서치
'''1. 탐색할 매개변수를 지정
   2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다
       -> 이 조합은 그리드 서치 객체에 저장됨
   3. 그리드 서치는 최상의 매개변수에서(교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해
      최종 모뎅르 훈련함 -> 이 모델도 그리드 서치 객체에 저장됨
'''

'1. 탐색할 매개변수를 지정\n   2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다\n       -> 이 조합은 그리드 서치 객체에 저장됨\n   3. 그리드 서치는 최상의 매개변수에서(교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해\n      최종 모뎅르 훈련함 -> 이 모델도 그리드 서치 객체에 저장됨\n'

In [28]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2, 100, 10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [29]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [32]:
#탐색할 매개변수의 간격을 설정한 근거가 없음, 어떻게 설정해야 할까?
#랜덤 서치 : 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달
from scipy.stats import uniform, randint # scipy는 수치 계산 전용 라이브러리

In [34]:
rgen = randint(0,10)
rgen.rvs(10)
ugen = uniform(0,1)
ugen.rvs(10)

array([0.71176117, 0.34779154, 0.85134199, 0.74047377, 0.08040047,
       0.33318747, 0.43129774, 0.39095319, 0.3123025 , 0.1356534 ])

In [36]:
''' 랜덤 서치에 randint과 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있다
    샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게 하는 것이 좋다'''
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25)} # min_samples_leaf는 리프 노드가 되기 위한 최소 샘플의 개수
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, 
                  n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fae90495df0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fae9044ff70>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fae90489100>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fae90489f70>},
                   random_state=42)

In [40]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
